In [1]:
#SUDOKO ANNELING
import numpy as np
import matplotlib.pyplot as plt
import random

# Define the Sudoku puzzle
initial_board = np.array([
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
])

def print_board(board):
    """ Prints the Sudoku board. """
    for row in board:
        print(' '.join(str(num) if num != 0 else '.' for num in row))
    print()
def objective_function(board):
    """ Calculates the number of conflicts in the board. """
    conflicts = 0
    for row in range(9):
        conflicts += len(board[row]) - len(set(board[row]))  # Row conflicts
    for col in range(9):
        conflicts += len(board[:, col]) - len(set(board[:, col]))  # Column conflicts
    for block_row in range(0, 9, 3):
        for block_col in range(0, 9, 3):
            block = board[block_row:block_row + 3, block_col:block_col + 3]
            conflicts += (9 - len(set(block.flatten())))  # Block conflicts
    return conflicts
def generate_neighbor(board, fixed_positions):
    """ Generates a neighbor by swapping two random values in the same row, excluding fixed positions. """
    new_board = np.copy(board)
    row = random.randint(0, 8)
    
    # Find the available columns (non-fixed positions) in the row
    available_cols = [col for col in range(9) if (row, col) not in fixed_positions]
    
    if len(available_cols) >= 2:
        col1, col2 = random.sample(available_cols, 2)
        new_board[row, col1], new_board[row, col2] = new_board[row, col2], new_board[row, col1]
    
    return new_board

def random_fill(board, fixed_positions):
    """ Fills the Sudoku board randomly while respecting the initial clues. """
    for row in range(9):
        # Get the set of numbers already in the row
        existing_numbers = set(board[row]) - {0}
        # Numbers that can be used to fill the row
        numbers_to_place = list(set(range(1, 10)) - existing_numbers)
        random.shuffle(numbers_to_place)
        
        for col in range(9):
            if board[row, col] == 0 and (row, col) not in fixed_positions:
                board[row, col] = numbers_to_place.pop()

def simulated_annealing(board, fixed_positions, initial_temp=1.0, cooling_rate=0.995, min_temp=0.01):
    """ Solves Sudoku using Simulated Annealing. """
    current_board = np.copy(board)
    best_board = np.copy(board)
    best_cost = objective_function(best_board)
    temp = initial_temp
    
    while temp > min_temp:
        neighbor_board = generate_neighbor(current_board, fixed_positions)
        current_cost = objective_function(current_board)
        neighbor_cost = objective_function(neighbor_board)
        
        if neighbor_cost < best_cost:
            current_board = neighbor_board
            best_board = neighbor_board
            best_cost = neighbor_cost
        elif np.random.rand() < np.exp((current_cost - neighbor_cost) / temp):
            current_board = neighbor_board
        
        temp *= cooling_rate
    
    return best_board

# Identify the fixed positions (non-zero initial values)
fixed_positions = [(r, c) for r in range(9) for c in range(9) if initial_board[r, c] != 0]

# Initial random filling to start
random_fill(initial_board, fixed_positions)

# Solve Sudoku using Simulated Annealing
solution_board = simulated_annealing(initial_board, fixed_positions)

print("Solved Sudoku:")
print_board(solution_board)
 



Solved Sudoku:
5 3 2 6 7 8 4 1 9
6 7 4 1 9 5 8 3 2
1 9 8 2 4 3 5 6 7
8 5 1 9 6 4 7 2 3
4 2 6 8 7 3 9 5 1
7 3 9 5 2 1 8 4 6
9 6 3 7 5 1 2 8 4
2 8 7 4 1 9 3 6 5
4 1 5 3 8 2 6 7 9

